In [2]:
from surprise import KNNBasic,Dataset,Reader
from collections import defaultdict
from operator import itemgetter
import heapq as hq
import os
import csv

In [12]:
#Load movie.csv and ratings.csv to return a dataset
def load_dataset():
    ratings_dataset = 0
    reader = Reader(line_format = 'user item rating timestamp', sep = ',', skip_lines=1)
    ratings_dataset = Dataset.load_from_file('ml-dataset-latest/ratings.csv',reader=reader)

    #Retrieve movie's title using it's ID
    movie_titles = {}
    with open('ml-dataset-latest/movies.csv', newline = '', encoding = 'ISO-8859-1') as movie_csv:
        reader_movie = csv.reader(movie_csv)
        next(reader_movie)
        for row in reader_movie:
            movie_id = int(row[0])
            movie_name = row[1]
            movie_titles[movie_id] = movie_name

    return(ratings_dataset,movie_titles)

dataset,movie_titles = load_dataset()

training_set = dataset.build_full_trainset()


        



